# Data loading 
Next cells are used to load the data we need.

In [65]:
import pandas as pd
import numpy as np 
import scipy.sparse as sps
import os
import sys
import matplotlib.pyplot as plt
sys.path.append('..')


In [66]:
#data which contains users (row), items(col) and implicit interaction (data)
dataset = pd.read_csv('data_train.csv')
dataset

,row,col,data
0,0,10080,1.0
1,0,19467,1.0
2,1,2665,1.0
3,1,7494,1.0
4,1,17068,1.0
...,...,...,...
113263,7945,2476,1.0
113264,7945,12319,1.0
113265,7945,21384,1.0
113266,7946,8699,1.0


In [67]:
unique_users, min_users, max_users = len(dataset.row.unique()), dataset.row.min(), dataset.row.max()
unique_items, min_items, max_items = len(dataset.col.unique()), dataset.col.min(), dataset.col.max()
print('users stats:',unique_users, min_users, max_users)
print('items stats:',unique_items, min_items, max_items)

users stats: 7947 0 7946
items stats: 24896 0 25974


In [68]:
users = dataset.row
items = dataset.col
data = dataset.data
URM_all = sps.coo_matrix((data, (users, items)))
URM_all = URM_all.tocsr() #fast row access -> fast access to users 
URM_all.shape

(7947, 25975)

In [69]:
items_per_user = np.ediff1d(URM_all.indptr) #tells in which position each of the rows begin
users_per_item = np.ediff1d(sps.csc_matrix(URM_all).indptr)
URM_all = sps.csr_matrix(URM_all)

In [70]:
items_per_user = np.sort(items_per_user) #sorting based on the single value. Losing reference to the user. 
users_per_item = np.sort(users_per_item)

In [71]:
ICM_df = pd.read_csv('data_ICM_title_abstract.csv')
ICM_df

,row,col,data
0,0,1185,1.015524
1,0,2507,0.459024
2,0,3534,0.227742
3,0,8766,0.501549
4,0,10862,0.297011
...,...,...,...
490686,25974,12554,0.963016
490687,25974,13003,0.104613
490688,25974,16236,0.118760
490689,25974,18797,0.363301


In [72]:
items = ICM_df.row
features = ICM_df.col
data = ICM_df.data
ICM_all = sps.coo_matrix((data, (items, features)))
ICM_all = ICM_all.tocsr() #fast row access -> fast access to users 
ICM_all.shape

(25975, 20000)

In [73]:
ICM_all = sps.csr_matrix(ICM_all)
features_per_item = np.ediff1d(ICM_all.indptr)

ICM_all = sps.csc_matrix(ICM_all)
items_per_feature = np.ediff1d(ICM_all.indptr)

ICM_all = sps.csr_matrix(ICM_all)

In [74]:
features_per_item = np.sort(features_per_item)
items_per_feature = np.sort(items_per_feature)

In [75]:
from Base.Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.85)
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])


In [174]:
from GraphBased.P3alphaRecommender import P3alphaRecommender
from GraphBased.RP3betaRecommender import RP3betaRecommender
from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender

recommender_alpha = P3alphaRecommender(URM_train)
recommender_alpha.fit(topK=500, alpha = 0.45, implicit = True)

recommender_beta_ICM = RP3betaRecommender(ICM_all.T)
recommender_beta_ICM.fit(topK=100, alpha=0.2, beta=0.5, implicit=False)
recommender_beta_ICM.URM_train = URM_train

recommeder_KNNItemCBF = ItemKNNCBFRecommender(URM_train, ICM_all)
recommeder_KNNItemCBF.fit(topK=100, shrink=400, feature_weighting = 'TF-IDF') #Vedere quale normalizzazione ha più senso dal punto di vista teorico

P3alphaRecommender: URM Detected 42 (0.53 %) cold users.
P3alphaRecommender: URM Detected 1961 (7.55 %) cold items.
RP3betaRecommender: URM Detected 2 (0.01 %) cold users.
RP3betaRecommender: URM Detected 25 (0.10 %) cold items.
ItemKNNCBFRecommender: URM Detected 42 (0.53 %) cold users.
ItemKNNCBFRecommender: URM Detected 1961 (7.55 %) cold items.
Similarity column 25975 ( 100 % ), 3487.60 column/sec, elapsed time 0.12 min


In [175]:

hybridrecommender_CBF = ItemKNNSimilarityHybridRecommender(URM_train, recommender_beta_ICM.W_sparse, recommeder_KNNItemCBF.W_sparse)
hybridrecommender_CBF.fit(topK=200, alpha = 0.9)

hybridrecommender = ItemKNNSimilarityHybridRecommender(URM_train, recommender_alpha.W_sparse, hybridrecommender_CBF.W_sparse)
hybridrecommender.fit(topK=550, alpha = 0.1)

evaluator_validation.evaluateRecommender(hybridrecommender)

ItemKNNSimilarityHybridRecommender: URM Detected 42 (0.53 %) cold users.
ItemKNNSimilarityHybridRecommender: URM Detected 1961 (7.55 %) cold items.
ItemKNNSimilarityHybridRecommender: URM Detected 42 (0.53 %) cold users.
ItemKNNSimilarityHybridRecommender: URM Detected 1961 (7.55 %) cold items.
EvaluatorHoldout: Processed 4973 ( 100.00% ) in 4.08 sec. Users per second: 1220


({10: {'ROC_AUC': 0.19676310808524836,
   'PRECISION': 0.041403579328372256,
   'PRECISION_RECALL_MIN_DEN': 0.16635243010670375,
   'RECALL': 0.16336976877469309,
   'MAP': 0.07522564168252065,
   'MRR': 0.14253500649539258,
   'NDCG': 0.11246290387465528,
   'F1': 0.06606419481813007,
   'HIT_RATE': 0.41403579328373213,
   'ARHR': 0.16334180766615955,
   'NOVELTY': 0.004894394786306075,
   'AVERAGE_POPULARITY': 0.09616700331041342,
   'DIVERSITY_MEAN_INTER_LIST': 0.987482963109399,
   'DIVERSITY_HERFINDAHL': 0.9987284394244909,
   'COVERAGE_ITEM': 0.4695283926852743,
   'COVERAGE_ITEM_CORRECT': 0.05039461020211742,
   'COVERAGE_USER': 0.6257707310934943,
   'COVERAGE_USER_CORRECT': 0.1946646533283,
   'DIVERSITY_GINI': 0.17172764341054259,
   'SHANNON_ENTROPY': 11.824777730310903}},
 'CUTOFF: 10 - ROC_AUC: 0.1967631, PRECISION: 0.0414036, PRECISION_RECALL_MIN_DEN: 0.1663524, RECALL: 0.1633698, MAP: 0.0752256, MRR: 0.1425350, NDCG: 0.1124629, F1: 0.0660642, HIT_RATE: 0.4140358, ARHR: 0

In [132]:
test_users = pd.read_csv('data_target_users_test.csv')
test_users

,user_id
0,0
1,1
2,2
3,3
4,4
...,...
7939,7942
7940,7943
7941,7944
7942,7945


In [26]:

recommender_beta_ICM = RP3betaRecommender(ICM_all.T)
recommender_beta_ICM.fit(topK=70, alpha=0.2, beta=0.5, implicit=False)
recommender_beta_ICM.URM_train = URM_all

recommeder_KNNItemCBF = ItemKNNCBFRecommender(URM_all, ICM_all)
recommeder_KNNItemCBF.fit(topK=100, shrink=300, feature_weighting = 'TF-IDF') #Vedere quale normalizzazione ha più senso dal punto di vista teorico


hybridrecommender_CBF = ItemKNNSimilarityHybridRecommender(URM_all, recommender_beta_ICM.W_sparse, recommeder_KNNItemCBF.W_sparse)
hybridrecommender_CBF.fit(topK=200, alpha = 0.9)


recommender_alpha = P3alphaRecommender(URM_all)
recommender_alpha.fit(topK=170, alpha = 0.45, implicit = True)

hybridrecommender = ItemKNNSimilarityHybridRecommender(URM_all, recommender_alpha.W_sparse, hybridrecommender_CBF.W_sparse)
hybridrecommender.fit(topK=200, alpha = 0.1)

evaluator_validation.evaluateRecommender(hybridrecommender)

user_id = test_users['user_id']
recommendations = hybridrecommender.recommend(user_id,cutoff = 10)


RP3betaRecommender: URM Detected 2 (0.01 %) cold users.
RP3betaRecommender: URM Detected 25 (0.10 %) cold items.
ItemKNNCBFRecommender: URM Detected 1079 (4.15 %) cold items.
Similarity column 25975 ( 100 % ), 3287.96 column/sec, elapsed time 0.13 min
ItemKNNSimilarityHybridRecommender: URM Detected 1079 (4.15 %) cold items.
P3alphaRecommender: URM Detected 1079 (4.15 %) cold items.
ItemKNNSimilarityHybridRecommender: URM Detected 1079 (4.15 %) cold items.
EvaluatorHoldout: Processed 4965 ( 100.00% ) in 4.49 sec. Users per second: 1105


In [27]:
for index in range(len(recommendations)):
    #print(element)
    recommendations[index]=np.array(recommendations[index])
    #print(type(element))
print(len(recommendations))

7944


In [28]:
test_users['item_list']= recommendations
#test_users['item_list'] =  test_users['item_list'].apply(lambda x: x.replace('[','').replace(']','')) 
test_users['item_list'] = pd.DataFrame([str(line).strip('[').strip(']').replace("'","") for line in test_users['item_list']])

#convert the string columns to int
#test_users['item_list'] = test_users['item_list'].astype(int)
test_users
test_users.to_csv('submission.csv', index=False)
